<h1>Preparing env</h1>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from collections import Counter
import math

In [ ]:
#uploading all data
PATH_DATA = './dataset'
tr_1 = pd.read_csv(os.path.join(PATH_DATA, 'transaction_1.csv'), index_col='client_id').sort_index()
tr_2 = pd.read_csv(os.path.join(PATH_DATA, 'transaction_2.csv'), index_col='client_id').sort_index()
tr_3 = pd.read_csv(os.path.join(PATH_DATA, 'transaction_3.csv'), index_col='client_id').sort_index()
tr_4 = pd.read_csv(os.path.join(PATH_DATA, 'transaction_4.csv'), index_col='client_id').sort_index()
tr_5 = pd.read_csv(os.path.join(PATH_DATA, 'transaction_5.csv'), index_col='client_id').sort_index()
tr_6 = pd.read_csv(os.path.join(PATH_DATA, 'transaction_6.csv'), index_col='client_id').sort_index()
transactions = pd.concat(
    [tr_1, tr_2, tr_3, tr_4, tr_5, tr_6]
)

mcc_codes = pd.read_csv(os.path.join(PATH_DATA, 'mcc_codes.csv'), sep=';', index_col='mcc_code')
gender_train = pd.read_csv(os.path.join(PATH_DATA, 'train.csv'), index_col='client_id')
trans_types = pd.read_csv('dataset/trans_types.csv')

# main settings about the style: 
main_template = 'plotly_dark'

<h1>Code<h1>

<h3>show dataframes in row form<h3>

In [ ]:
print('trans_types:')
display(trans_types)

print('transactions:')
display(transactions)

print('gender_train:')
display(gender_train)

print('mcc_codes:')
display(mcc_codes)


<h3>records of male and female transfers<h3>

In [ ]:
def change_bool_to_names(gender_value:int) -> str:
    if gender_value:
        return 'male'
    return 'female'

In [ ]:
# combine them into one dataframe with 2 columns
# client_id & gender 
genders_id_group = pd.DataFrame(transactions.index.copy()).set_index('client_id')
genders_id_group = genders_id_group.join(gender_train)
genders_id_group = genders_id_group.loc[:, ~genders_id_group.columns.str.contains('^Unnamed')]
genders_id_group = genders_id_group[~genders_id_group.index.duplicated(keep='first')]


genders_id_group = genders_id_group.reset_index().groupby('gender')['client_id'].count().reset_index()
genders_id_group['gender'][0] = 'female'
genders_id_group['gender'][1] = 'male'

#make simple bar chart using this data

fig_genders_id_group = px.bar(genders_id_group,
    x = 'gender',
    y = 'client_id', 
    template=main_template, 
    labels={'client_id': 'quantity'}, 
    title='how many men and women in dataset', 
    color='gender'
)

fig_genders_id_group.show()

<h3>time period covered in the data</h3>

In [ ]:
# # it takes about minute, on m2
# def exclude_trans_time_ext(n):
#     return (n>0)*(n)

# period_covered_sl = {}

# prev_index = transactions.index[0]
# temp_max = -1
# temp_min = 10**5
# for index, row in transactions.sort_index().iterrows():
#     data = int(row['trans_time'].split()[0])
#     if index == prev_index:
#         temp_max = max(temp_max, data)
#         temp_min = min(temp_min, data)
#     else:
#         period_covered_sl[index]=temp_max-temp_min
#         prev_index = index
#         temp_min = 10**5
#         temp_max = -1


# df_period_covered = pd.DataFrame(list(period_covered_sl.items()), columns=['client_id', 'count'])
# df_period_covered['count'] = df_period_covered['count'].apply(exclude_trans_time_ext)

In [ ]:

# fig_period_covered = px.histogram(df_period_covered,
#                                 x = 'count', 
#                                 template=main_template, 
#                                 title = 'Spending rate')
# fig_period_covered.update_layout(xaxis_title = 'period of spendings in days', 
#                                  yaxis_title = 'users count', 
#                                  title_x = 0.45,
#                                  bargap = 0.04)

# fig_period_covered.show()

# fig_cor_period_covered = px.histogram(df_period_covered.loc[df_period_covered['count']>430],
#                                 x = 'count', 
#                                 template=main_template, 
#                                 title = 'Adjustment Spending rate')
# fig_cor_period_covered.update_layout(xaxis_title = 'period of spendings in days', 
#                                  yaxis_title = 'users count', 
#                                  title_x = 0.45,
#                                  bargap = 0.2)

# fig_cor_period_covered.show()

<h3>mean, std, e.t.c</h3>

In [ ]:
# lets find how many women and men spend for given period:
# merge transactons and genders by client_id:
extended_transactions = pd.merge(transactions, gender_train, left_index=True, right_index=True)
#slit men's and women's transactions
women_spendings = extended_transactions.loc[(gender_train['gender'] == 0) & (extended_transactions['amount'] < 0)]
men_spendings = extended_transactions.loc[(gender_train['gender'] == 1) & (extended_transactions['amount'] < 0)]

#group all spends of each users
gr_women_spendings = women_spendings.groupby(level=0)['amount'].sum().reset_index().set_index('client_id')
gr_men_spendings = men_spendings.groupby(level=0)['amount'].sum().reset_index().set_index('client_id')

# histogram about their spendings
n = 60
array_of_closest = [x*(1_500_000/n) for x in range(n)]

#Python Code to Find closest number in Sorted array using recursion
def find_closest_recursive(arr, left, right, target):
	# base case: when there is only one element in the array
	if left == right:
		return arr[left]

	# calculate the middle index
	mid = (left + right) // 2

	# recursively search the left half of the array
	left_closest = find_closest_recursive(arr, left, mid, target)

	# recursively search the right half of the array
	right_closest = find_closest_recursive(arr, mid + 1, right, target)

	# compare the absolute differences of the closest elements in the left and right halves
	if abs(left_closest - target) <= abs(right_closest - target):
		return left_closest
	else:
		return right_closest


gr_men_spendings_cor = gr_men_spendings.reset_index()['amount'].abs().tolist()
gr_men_spendings_cor = [find_closest_recursive(array_of_closest, 0, n-1, x) for x in gr_men_spendings_cor]

gr_men_spendings_cor = list(Counter(gr_men_spendings_cor).items())
gr_men_spendings_cor = pd.DataFrame(gr_men_spendings_cor).rename(columns={0:'amount', 1:'count'})

gr_men_spendings_cor = gr_men_spendings_cor.sort_values('amount')
gr_men_spendings_cor = gr_men_spendings_cor.head(len(gr_men_spendings_cor)-1)
gr_men_spendings_cor['gender'] = ['male']*len(gr_men_spendings_cor)

# same for women
gr_women_spendings_cor = gr_women_spendings.reset_index()['amount'].abs().tolist()
gr_women_spendings_cor = [find_closest_recursive(array_of_closest, 0, n-1, x) for x in gr_women_spendings_cor]

gr_women_spendings_cor = list(Counter(gr_women_spendings_cor).items())
gr_women_spendings_cor = pd.DataFrame(gr_women_spendings_cor).rename(columns={0:'amount', 1:'count'})

gr_women_spendings_cor = gr_women_spendings_cor.sort_values('amount')
gr_women_spendings_cor = gr_women_spendings_cor.head(len(gr_women_spendings_cor)-1)
gr_women_spendings_cor['gender'] = ['female']*len(gr_women_spendings_cor)


In [ ]:
avg_men_spendings = round(abs(gr_men_spendings['amount'].mean()), 2)
avg_women_spendings = round(abs(gr_women_spendings['amount'].mean()), 2)

bar_c_of_avg_spendings = px.bar(
            x = [avg_men_spendings, avg_women_spendings],
            y = ['men', 'women'],
            orientation='h',
            title = "AVG men's and women's spendings: (sum over a period)", 
            labels={ "x": "quantity in millions of rubles",
                     "y": "sex"},
                     template=main_template,
                     )

bar_c_of_avg_spendings.show()

In [ ]:
fig_box_gr_men_spendigns = px.box(gr_men_spendings['amount'].abs(), 
             template=main_template, 
             title = "Box plot, men's spendings", 
             y = 'amount', 
             points='all')

fig_box_gr_men_spendigns.show()

In [ ]:
fig_box_gr_men_spendigns_cor = px.box(gr_men_spendings_cor, y='amount', template=main_template, 
             title = "Adjusted data about men's spendings over the period (less then 1.5 M rub)")

fig_box_gr_men_spendigns_cor.show()


<h1>Data details<h1>

In [ ]:
# normalize the spendings period to 30 days
# mo_extended_transactions = extended_transactions.reset_index()

def crutch_to_normalise_sorting(value: str):
    return (len(value.split()[0])>2)*value

mo_extended_transactions = extended_transactions
mo_extended_transactions['trans_time'] = mo_extended_transactions['trans_time'].apply(crutch_to_normalise_sorting)
mo_extended_transactions = mo_extended_transactions.sort_values(
    by=['client_id', 'trans_time'], ascending=[False, False])


In [ ]:
gr_women_spendings_cor['gender'] = ['female']*len(gr_women_spendings_cor)

fig_men_spendings_bar = px.bar(
		gr_men_spendings_cor,
		x = 'amount', 
		y = 'count', 
		template=main_template, 
		color='amount', 
		title = "distribution chart of men's spendings",
		labels={'count':'amount of people'}
						)
fig_women_spendings_bar = px.bar(
		gr_women_spendings_cor,
		x = 'amount', 
		y = 'count', 
		template=main_template, 
		color='amount',
		title = "distribution chart of women's spendings",
		labels={'count':"amount of people"}
						)

fig_men_spendings_bar.show()
fig_women_spendings_bar.show()

merge_spendings_cor = pd.concat([gr_men_spendings_cor, gr_women_spendings_cor])

fig_spendings_merge = px.line(merge_spendings_cor, 
							  color='gender', 
							  x = 'amount', 
							  y = 'count', 
							  template=main_template, 
							  title="distribution chart of men's and women's spendings",
							  labels={
									'1':'men',
				 					'amount':'spends in rubles',
									'count':'amount of people'
										})

fig_spendings_merge.show()




In [ ]:
# lets visualise all men's and women's spendings time information:

partical_men_sp_data = men_spendings[['trans_time', 'amount']].copy()
partical_women_sp_data = women_spendings[['trans_time', 'amount']].copy()

def sep_time_and_data(trans_time):
    return trans_time[-8:-6]

# def mcc_value_s(mcc_code):
#     return mcc_means[mcc_code]['mcc_description']

def cor_amount(amount):
    return (abs(amount)<130_000) * abs(amount)


partical_men_sp_data['trans_time'] = (partical_men_sp_data['trans_time'].apply(sep_time_and_data))
# partical_men_sp_data['mcc_code'] = partical_men_sp_data['mcc_code'].apply(mcc_value_s)
partical_men_sp_data['amount'] = partical_men_sp_data['amount'].apply(cor_amount)
cut_partical_men_sp_data = partical_men_sp_data.head(100_000)
partical_men_sp_data = partical_men_sp_data.sort_values('trans_time')


partical_women_sp_data['trans_time'] = partical_women_sp_data['trans_time'].apply(sep_time_and_data)
# partical_women_sp_data['mcc_code'] = partical_women_sp_data['mcc_code'].apply(mcc_value_s)
partical_women_sp_data['amount'] = partical_women_sp_data['amount'].apply(cor_amount)
cut_partical_women_sp_data = partical_women_sp_data.head(100_000)
partical_women_sp_data = partical_women_sp_data.sort_values('trans_time')


In [ ]:
time_spendings_men = px.histogram(partical_men_sp_data, x = 'trans_time', y = 'amount',
                                    template=main_template,
                                    labels={'trans_time':'transactions time'},
                                    title="sum of men's transactions on each hour",
                                    color_discrete_sequence=px.colors.qualitative.Set1)
time_spendings_men.update_layout(yaxis_title = 'sum of transaction on eeach hour')
time_spendings_men.show()

time_spendings_women = px.histogram(partical_women_sp_data, x = 'trans_time', y = 'amount',
                                    template=main_template,
                                    labels={'trans_time':'transactions time'},
                                    title="sum of women's transactions on each hour", 
                                    color_discrete_sequence=px.colors.qualitative.Set3)
time_spendings_women.update_layout(yaxis_title = 'sum of transaction on eeach hour')
time_spendings_women.show()

In [ ]:
# show how much operation they perform on each hour
plank = 1000
def amount_transofrm(value):
    return int(value>=plank)


count_trans_men = partical_men_sp_data.copy()
count_trans_men['amount'] = count_trans_men['amount'].apply(amount_transofrm)

count_trans_women = partical_women_sp_data.copy()
count_trans_women['amount'] = count_trans_women['amount'].apply(amount_transofrm)

fig_count_trans_per_hounr_men = px.histogram(
            count_trans_men, 
            x = 'trans_time', 
            y = 'amount', 
            title=f'how many operations greater then {plank} rub men perform in an hour', 
            template=main_template,
            color_discrete_sequence=px.colors.qualitative.Set1, 
            labels={'trans_time':'transactions time'})
fig_count_trans_per_hounr_men.update_layout(yaxis_title="how many operations men perform in an hour")

fig_count_trans_per_hounr_men.show()

fig_count_trans_per_hounr_women = px.histogram(
            count_trans_women, 
            x = 'trans_time', 
            y = 'amount', 
            title=f'how many operations greater then {plank} rub women perform in an hour ', 
            template=main_template,
            color_discrete_sequence=px.colors.qualitative.Set3, 
            labels={'trans_time':'transactions time'})
fig_count_trans_per_hounr_women.update_layout(yaxis_title="how many operations men perform in an hour")

fig_count_trans_per_hounr_women.show()


In [ ]:
#collecting men's and women's mean spendings by category 
mean_mcc_men = men_spendings.groupby('mcc_code')['amount'].sum().reset_index().set_index('mcc_code')
mean_mcc_women = women_spendings.groupby('mcc_code')['amount'].sum().reset_index().set_index('mcc_code')

mcc_means = mcc_codes.to_dict('index')

In [ ]:
# get statistics on the 15 most popular expenses of men and women
top10_men_sum = abs(mean_mcc_men.sort_values('amount').head(10)).reset_index()
top10_women_sum = abs(mean_mcc_women.sort_values('amount').head(10)).reset_index()

# visualise it:

top10_men_mcc_names = [mcc_means[x]['mcc_description'] for x in top10_men_sum['mcc_code'].tolist()]

fig_top10_men = px.bar(top10_men_sum, y = 'amount', x = [x for x in range(10)], 
            title="Top 10 men's spending categories", 
            labels={'y':'spending categories'}, 
            template=main_template, 
            color=top10_men_mcc_names)

fig_top10_men.update_layout(legend_orientation="h")

fig_top10_men.show()

top10_women_mcc_names = [mcc_means[x]['mcc_description'] for x in top10_women_sum['mcc_code'].tolist()]

fig_top10_women = px.bar(top10_women_sum, y = 'amount', x = [x for x in range(10)], 
            title="Top 10 women's spending categories", 
            labels={'y':'spending categories'}, 
            template=main_template, 
            color=top10_women_mcc_names)

fig_top10_women.update_layout(legend_orientation="h")

fig_top10_women.show()

In [ ]:


city_distr_men = extended_transactions.loc[(extended_transactions['gender']==1) & (extended_transactions['amount']<0)].reset_index(
).groupby(['client_id', 'trans_city'])['amount'].sum().reset_index()

city_distr_men['amount'] = city_distr_men['amount'].abs()

city_distr_women = extended_transactions.loc[(extended_transactions['gender']==0) & (extended_transactions['amount']<0)].reset_index(
).groupby(['client_id', 'trans_city'])['amount'].sum().reset_index()

city_distr_women['amount'] = city_distr_men['amount'].abs()

fig_distr_city_men = px.box(city_distr_men, x = 'trans_city', y = 'amount', 
             template=main_template, 
             title = 'distribution of spending by city among men (relevant part)', 
             labels={'amount':'spending data', 'trans_city':'city name'})
fig_distr_city_men.update_layout( 
    yaxis=dict(
        range=[0,2*10**6]))

fig_distr_city_men.show()
fig_distr_city_women = px.box(city_distr_women, x = 'trans_city', y = 'amount', 
             template=main_template, 
             title = 'distribution of spending by city among women (relevant part)', 
             labels={'amount':'spending data', 'trans_city':'city name'})
fig_distr_city_women.update_layout( 
    yaxis=dict(
        range=[0,2*10**6]))
fig_distr_city_women.show()


<h3>What time people make the most expensive purchases<h3>

In [ ]:
partical_men_sp_data = partical_men_sp_data.groupby('trans_time')['amount'].median().reset_index()
partical_women_sp_data = partical_women_sp_data.groupby('trans_time')['amount'].median().reset_index()

partical_men_sp_data['gender']='male'
partical_women_sp_data['gender']='female'

merge_time_sp_data = pd.concat([partical_men_sp_data, 
                                partical_women_sp_data])

fig_bar_top_time = px.bar(merge_time_sp_data, 
                          y = 'amount', 
                          x = 'trans_time', 
                          template = main_template, 
                          title = 'The average amount of money spent per hour (median)', 
                          labels={'trans_time':'time'}, 
                          color='gender', 
                          barmode='group')

fig_bar_top_time.show()

<h3>who pays in restaurants and when<h3>

In [ ]:
restaurant_codes = [5812,5813,5814]
restaurant_data = extended_transactions.loc[
    extended_transactions['mcc_code'].isin(restaurant_codes)]

def sep_time_and_data(trans_time):
    return (trans_time[-8:-6])

restaurant_data['trans_time'] = (restaurant_data['trans_time'].apply(sep_time_and_data))
restaurant_data['amount'] = restaurant_data['amount'].abs()

rest_woman_temp = restaurant_data.loc[restaurant_data['gender']==0]
rest_man_temp = restaurant_data.loc[restaurant_data['gender']==1]

rest_woman_temp = rest_woman_temp.groupby('trans_time')['amount'].sum().reset_index()
rest_man_temp = rest_man_temp.groupby('trans_time')['amount'].sum().reset_index()

rest_woman_temp['gender']='female'
rest_man_temp['gender']='male'

count_rest_data = pd.concat([rest_woman_temp, 
                                rest_man_temp])



In [ ]:

fig_bar_rest_time_sum = px.bar(count_rest_data, 
             y='amount', x = 'trans_time', 
             color='gender', 
             template=main_template, 
             barmode='relative', 
             labels={'trans_time':'hour'}, 
             title = 'Amount spent on restaurants by men and women by hour (sum, barmode=relative)')

fig_bar_rest_time_sum.show()

In [ ]:
nan_terms = restaurant_data['term_id'].isna().sum()

fig_terms_trans_pie = px.pie(
    values=[nan_terms, len(restaurant_data)-nan_terms], 
    names=['by terminal', 'by transfer'],
    template=main_template, 
    title='Ratio of the number of transactions paid via terminal and transfer', 
    hole=0.3
    )
fig_terms_trans_pie.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=15,
                  marker=dict(line=dict(color='#ffffff', width=0.5)))

fig_terms_trans_pie.show()

In [ ]:
def check_terminal_present(term_id: str) -> str:
    if len(str(term_id))!=3:
        return 'By terminal'
    return 'By transfer'

restaurant_data['term_id'] = restaurant_data['term_id'].apply(check_terminal_present)

restaurant_data_women = restaurant_data.loc[restaurant_data['gender']==0]
restaurant_data_men = restaurant_data.loc[restaurant_data['gender']==1]

restaurant_data_men = restaurant_data_men.groupby('term_id')['amount'].count().reset_index()
restaurant_data_women = restaurant_data_women.groupby('term_id')['amount'].count().reset_index()

restaurant_data_men['gender'] = 'male'
restaurant_data_women['gender'] = 'female'


restaurant_data_4_lines = pd.concat([restaurant_data_men, restaurant_data_women])

In [ ]:
fig = px.bar(
    restaurant_data_4_lines, 
    x = 'gender', 
    y = 'amount', 
    color='term_id', 
    barmode='group', 
    template=main_template,
    labels={'term_id':'type'},
    title='Who and how to make payments.' 
)

fig.show()

In [ ]:
fig_box_city_rest = px.box(
    restaurant_data, 
    x = 'trans_city', 
    y = 'amount', 
    color='gender', 
    template=main_template,
    labels={'trans_city':'city',
            '1':'male', 1:'male', 
            '0':'female', 0:'female'},
    title = 'Spending on restaurants in different Russian cities (1-male, 0-female)' 

)
fig_box_city_rest.update_layout( 
    yaxis=dict(
        range=[0,1000]))
fig_box_city_rest.show()

<h1>The Penza anomaly<h1>

<h3>overview
<h3>

In [ ]:
penza_anomaly_data = transactions.loc[
    transactions.index=='a288a664b343fb3b2ce8fc48ccfa328b']

penza_anomaly_spend = penza_anomaly_data.loc[penza_anomaly_data['amount']<0]
penza_anomaly_income = penza_anomaly_data.loc[penza_anomaly_data['amount']>0]

penzas_median = transactions.loc[transactions['trans_city'] == 'Penza']
penzas_median = penzas_median.groupby(level=0)['amount'].sum().reset_index()['amount'].median()


fig_penza_overview_income_spend = px.bar(
    x = ['Expenses', 'Income', 'Median value for Penza'], 
    y = [abs(penza_anomaly_spend['amount'].sum()), 
         penza_anomaly_income['amount'].sum(), 
         abs(penzas_median)],
    template=main_template,
    title = 'Scale of the anomaly', 
    labels= {'x':'', 'y': 'amount in rubles'}
)
fig_penza_overview_income_spend.show()

In [ ]:
def month_determ(data: str) -> int:
    return int(data.split()[0])//30 - 2

penza_data_month_spend = penza_anomaly_spend.copy()
penza_data_month_spend['trans_time'] = penza_data_month_spend['trans_time'].apply(month_determ)
penza_data_month_spend_gr = penza_data_month_spend.groupby('trans_time')['amount'].sum().reset_index()
penza_data_month_spend_gr['amount'] = penza_data_month_spend_gr['amount'].abs()
penza_data_month_spend_gr['type'] = 'expenses'

penza_data_month_income = penza_anomaly_income.copy()
penza_data_month_income['trans_time'] = penza_data_month_income['trans_time'].apply(month_determ)
penza_data_month_income_gr = penza_data_month_income.groupby('trans_time')['amount'].sum().reset_index()
penza_data_month_income_gr['type'] = 'income'

gr_penza_month = pd.concat([penza_data_month_income_gr, penza_data_month_spend_gr])

gr_penza_month = gr_penza_month.sort_values('trans_time')


In [ ]:
fig_penza_bar_month = px.bar(
    gr_penza_month, 
    y = 'amount', 
    x = 'trans_time',
    color='type',
    barmode='group', 
    template = main_template, 
    title = 'Duration of the anomaly observation',
    labels={'trans_time': 'observed month'}
)

fig_penza_bar_month.show()
# print(penza_anomaly_data['trans_city'].unique())
# only Penza


In [ ]:
ext_penza = penza_anomaly_data.copy()
ext_penza = ext_penza.reset_index().drop(columns=['client_id'])
ext_penza['type'] = np.where(ext_penza['amount'] > 0, 'income', 'expenses')
ext_penza['amount'] = ext_penza['amount'].abs()

def tranform_to_onlyDays(data:str) -> int:
    return int(data.split()[0])

four_month_penza = ext_penza.copy()
four_month_penza['trans_time'] = four_month_penza['trans_time'].apply(tranform_to_onlyDays)
four_month_penza = four_month_penza.loc[four_month_penza['trans_time']<=210]
four_month_penza = four_month_penza.loc[four_month_penza['trans_time']>=180]

four_month_penza_sped = four_month_penza.loc[four_month_penza['type']=='expenses']
four_month_penza_income = four_month_penza.loc[four_month_penza['type']=='income']

four_month_penza_sped = four_month_penza_sped.groupby('trans_time')['amount'].sum().reset_index()
four_month_penza_income = four_month_penza_income.groupby('trans_time')['amount'].sum().reset_index()

four_month_penza_sped['type'] = 'expenses'
four_month_penza_income['type'] = 'income'

four_month_penza = pd.concat([four_month_penza_sped, 
                              four_month_penza_income])

def four_month_clear_days(data: str) -> int:
    return data-180

four_month_penza['trans_time'] = four_month_penza['trans_time'].apply(four_month_clear_days)

In [ ]:
fig_penza_hist_days = px.bar(
    four_month_penza, 
    x = 'trans_time', 
    y = 'amount', 
    color='type', 
    template=main_template, 
    barmode='group', 
    title = "Distribution of expenditure and income by days of the fourth month", 
    labels={'trans_time':'day'}
)

fig_penza_hist_days.show()

<h3>Categories</h3>

In [ ]:
penza_times = ext_penza.copy()
def mcc_code_means(code: str) -> str:
    return mcc_codes.loc[code]['mcc_description']

ext_penza['mcc_code'] = ext_penza['mcc_code'].apply(mcc_code_means)


In [ ]:
df = ext_penza.groupby('mcc_code')['amount'].sum().reset_index()
df
fig_top3_penza_spend = px.bar(
    ext_penza.loc[ext_penza['type'] == 'expenses'].groupby(
        'mcc_code')['amount'].sum().reset_index().sort_values('amount', ascending=False),
        y = 'amount', x = [x+1 for x in range(3)], 
        template=main_template,
        color='mcc_code', 
        labels={'x':'', 'mcc_code':'category'}, 
        title = 'Income categories'
        )

fig_top3_penza_spend.show()

fig_top3_penza_income = px.bar(
    ext_penza.loc[ext_penza['type'] == 'income'].groupby(
        'mcc_code')['amount'].sum().reset_index().sort_values('amount', ascending=False),
        y = 'amount', x = ['1', '2'], 
        template=main_template,
        color='mcc_code', 
        labels={'x':'', 'mcc_code':'category'}, 
        title = 'Spending categories'
        )

fig_top3_penza_income.show()

<h3>ATM's</h3>

In [ ]:
penza_term = penza_anomaly_spend.groupby('term_id')['amount'].sum().reset_index()
penza_term['amount'] = penza_term['amount'].abs()
fig_penza_term = px.bar(penza_term, 
             x ='term_id', 
             y = 'amount', 
             template=main_template, 
             labels={'term_id':'terminal id'}, 
             title = 'What terminals she used to withdraw money from')


fig_penza_term.show()

<h3>anomaly time</h3>

In [ ]:
def sep_to_hours_int(data: str) -> int:
    return int(data[-8:-6])

penza_anomaly_spend['trans_time'] = penza_anomaly_spend['trans_time'].apply(sep_to_hours_int)
penza_anomaly_spend = pd.concat(
    [penza_anomaly_spend, 
     pd.DataFrame([[x, 0] for x in range(24)],
                  columns=['trans_time', 'amount'])])

penza_anomaly_spend = penza_anomaly_spend.groupby('trans_time')['amount'].sum().reset_index().sort_values('trans_time')
penza_anomaly_spend['amount'] = penza_anomaly_spend['amount'].abs()
penza_anomaly_spend['type'] = 'expenses'


penza_anomaly_income['trans_time'] = penza_anomaly_income['trans_time'].apply(sep_to_hours_int)
penza_anomaly_income = pd.concat(
    [penza_anomaly_spend, 
     pd.DataFrame([[x, 0] for x in range(24)],
                  columns=['trans_time', 'amount'])])

penza_anomaly_income = penza_anomaly_income.groupby('trans_time')['amount'].sum().reset_index().sort_values('trans_time')
penza_anomaly_income['type'] = 'income'

penza_times = pd.concat([penza_anomaly_income, penza_anomaly_spend])

In [ ]:
fig_penza_times = px.bar(
    penza_times, 
    x = 'trans_time', 
    y = 'amount', 
    color='type', 
    barmode='group', 
    title = 'At what time she spent and received the money',
    labels={'trans_time':'hour'}, 
    template=main_template
)

fig_penza_times.update_xaxes(minor=dict(ticklen=6, tickcolor="white"))

fig_penza_times.show()

<h2>Car owners</h2>

In [ ]:
codes_car = [5532,5533,5541,5542,7523,7531,7538]

car_owners = extended_transactions.loc[extended_transactions['mcc_code'].isin(codes_car)]
car_owners = set(car_owners.index.values.tolist())
tr_cars = extended_transactions.loc[extended_transactions.index.isin(car_owners)]

car_genders = gender_train.loc[gender_train.index.isin(car_owners)].drop(['Unnamed: 0'], axis = 1).reset_index()
car_genders = car_genders.groupby('gender')['client_id'].count().reset_index()
car_genders['gender'] = car_genders['gender'].apply(change_bool_to_names)

display(tr_cars)

In [ ]:
fig_car_genders_pie = px.pie(
    car_genders, 
    values='client_id',
    names='gender',
    template=main_template, 
    title = 'Gender distribution among car owners',
    hole=0.6
)

fig_car_genders_pie.show()

<body>Let's look at their income compared to those who don't have a car<body>

In [ ]:
no_cars_income = extended_transactions.loc[
    (extended_transactions.index.isin(car_owners)==False) & (extended_transactions['amount']>0)]
no_cars_gr_inc = no_cars_income.groupby(level=0)['amount'].sum().reset_index()
no_cars_gr_inc['car'] = 'carless'

cars_income = tr_cars.loc[tr_cars['amount']>0]
cars_gr_inc = cars_income.groupby(level=0)['amount'].sum().reset_index()
cars_gr_inc['car'] = 'have car'

merge_cars_inc = pd.concat([cars_gr_inc,no_cars_gr_inc]).set_index('client_id')

<body>Visualise this as a box plot<body>


In [ ]:
fig_car_income_box = px.box(
    merge_cars_inc, 
    x = 'car', 
    y = 'amount', 
    color='car', 
    title = 'Comparison of incomes of people with and without a car', 
    template=main_template, 
    labels={'car':'type'}
)

fig_car_income_box.update_layout(yaxis=dict(range=[0,1.5*10**6]))

fig_car_income_box.show()

<body>
Let's take a look at when people commit to medical spending. Maybe there's a seasonality there 
<body>

In [ ]:
# get all medicine spendings transactions
codes_medicine = [5912,8071,8099,8062,8011,5122]
medicine_spendings = extended_transactions.loc[
    (extended_transactions['mcc_code'].isin(codes_medicine)) &
    (extended_transactions['amount']<0)]

# remove time data and group them by week for clarity. Believe me, a 450-day chart is overnoisy
def keep_only_weeks_int(trans_time: str) -> int:
    return (int(trans_time.split()[0])//7)

medicine_spendings['trans_time'].replace('', np.nan, inplace=True)
medicine_spendings.dropna(subset=['trans_time'], inplace=True)

    
medicine_spendings['trans_time'] = medicine_spendings['trans_time'].apply(keep_only_weeks_int)

# remove the oversized transactions so they don't spoil the picture:
def remove_ext_medicine(amount: float) -> float:
    return (amount>-10_000)*amount

medicine_spendings['amount'] = medicine_spendings['amount'].apply(remove_ext_medicine)

#group by day number:
medicine_spendings_gr = medicine_spendings.groupby('trans_time')['amount'].sum().abs().reset_index().sort_values('trans_time')
# let's delete the last incomplete week so that the graph doesn't fall at the end 
medicine_spendings_gr = medicine_spendings_gr.head(len(medicine_spendings_gr)-1)

#abs for non-grooped dataframe (used for scatter and box plots)
medicine_spendings['amount'] = medicine_spendings['amount'].abs()    


In [ ]:
fig_medicine_line = px.line(
    medicine_spendings_gr,
    x = 'trans_time', #day number
    y = 'amount', 
    template=main_template, 
    title='Spending on medicine in 450 days',
    labels={'trans_time':'week number'}
)   

fig_medicine_line.show()


In [ ]:
fig_medicine_scatter = px.scatter(
    medicine_spendings, 
    x = 'trans_time', 
    y = 'amount', 
    template=main_template, 
    title='Spending on medicine in 450 days',
    labels={'trans_time':'week number'}, 
    color='mcc_code'
)
fig_medicine_scatter.show()

In [ ]:
#preparing dataframe for ulta_mega_super_combined_line_graph

medicine_mcc_gr = medicine_spendings.groupby(['trans_time', 'mcc_code'])['amount'].sum().reset_index().sort_values('trans_time')
medicine_mcc_gr = medicine_mcc_gr.head(len(medicine_mcc_gr)-4)

medicine_mcc_gr['mcc_code'] = medicine_mcc_gr['mcc_code'].apply(mcc_code_means)
medicine_mcc_gr=medicine_mcc_gr.sort_values(['trans_time', 'mcc_code'])

In [ ]:
fig_medicine_line_pro = px.line(
    medicine_mcc_gr,
    x = 'trans_time',
    y = 'amount', 
    color='mcc_code',
    template=main_template, 
    title='Spending on medicine in 450 days', 
    labels={'trans_time':'week number', 
            'mcc_code':'category:'},
)
fig_medicine_line_pro.update_layout(
    legend_orientation='h')
fig_medicine_line_pro.update_layout(
    legend_x = 0, 
    legend_y = -0.25,
    xaxis_type = "category",
    margin_r = 40,
    legend_title_text = "Category:",
)


fig_medicine_line_pro.show()

<body>
Let’s try to find a correlation between a person’s income and the frequency of their spending on medicine.
</body>


In [ ]:
def remove_income_extremums(amount: float) -> float:
    return (amount<10**6)*amount

def genders_to_names(gender_value: int) -> str:
    if gender_value:
        return 'male'
    return 'female'

In [ ]:
users_total_spend = extended_transactions.loc[extended_transactions['amount']<0].reset_index().groupby(
    ['client_id', 'gender'])['amount'].sum().abs().reset_index().set_index('client_id').sort_values('amount')


med_users_total_spent = medicine_spendings.reset_index().groupby(
    ['client_id', 'gender', 'mcc_code']
)['amount'].sum().reset_index().set_index('client_id').sort_index()

med_income_spend_correl = pd.merge(
    med_users_total_spent,
    users_total_spend, 
    right_index=True, 
    left_index=True
).drop(columns=['gender_y']).rename(
    columns={
        'gender_x':'gender', 
        'amount_x':'spent',
        'amount_y':'income'
        })

# rename genders: from 0-1 to mele-female
med_income_spend_correl['gender'] = np.where(
    med_income_spend_correl['gender'], 'male', 'female')

# Let’s remove the extremes, as they prevent us from understanding the general trend
med_income_spend_correl = med_income_spend_correl.loc[
    (med_income_spend_correl['income']<5_000_000) &
    (med_income_spend_correl['spent']<100_000)
]

In [ ]:
fig_med_income_scatter = px.scatter(
    med_income_spend_correl,
    x = 'income', 
    y = 'spent', 
    color='gender', 
    title='The dependence of spending on medicine on total income', 
    template=main_template,
    log_x=False,
)

fig_med_income_scatter.show()

<h4>
Let’s try to make this graph more illustrative.
<h4>

grouping by income (rounding), take spent median

In [ ]:
# grouping by income (rounding), take spent median

def round_income(income_value: float) -> int:
    return (income_value//15_000) * 15_000

# round users income:
med_income_spend_correl['income'] = med_income_spend_correl['income'].apply(round_income)

# grouping by income:
med_income_spend_correl = med_income_spend_correl.reset_index().groupby(
    ['income', 'gender', 'mcc_code']
)['spent'].median().reset_index().sort_values('income')

# change mcc codes to names:

med_income_spend_correl['mcc_code'] = med_income_spend_correl['mcc_code'].apply(mcc_code_means)

# cut the income range:
med_income_spend_correl = med_income_spend_correl.loc[med_income_spend_correl['income']<10**6]

In [ ]:
fig_medicine_income_bar_pro = px.bar(
    med_income_spend_correl,
    x = 'income', 
    y = 'spent', 
    color='mcc_code', 
    title='The dependence of spending on medicine on total income', 
    template=main_template, 
)
fig_medicine_income_bar_pro.update_layout(
    legend_orientation='h', 
    legend_y = -0.25, 
    legend_title = 'Category:'
)
fig_medicine_income_bar_pro.add_trace(
    go.Line(
        x = med_income_spend_correl.groupby(['income'])['spent'].sum().reset_index()['income'],
        y = med_income_spend_correl.groupby(['income'])['spent'].sum().reset_index()['spent'], 
        name = 'category sum'
    )
)

fig_medicine_income_bar_pro.show()

<h1>
Travelling
<h1>

Let's look at the transactions related to leisure. Such as: rental of cars, railway and air transportation, hotels, etc.

In [ ]:
# travelling codes
codes_trevel = [3000,3351,3501,4112,4411,4511,4722,7011,7512,7991]

trev_transactions = extended_transactions.loc[
    extended_transactions['mcc_code'].isin(codes_trevel)
]
trev_transactions = trev_transactions.loc[
    trev_transactions['amount']<0
]
trev_transactions['amount'] = trev_transactions['amount'].abs()

trev_transactions['trans_time'].replace('', np.nan, inplace=True)
trev_transactions.dropna(subset=['trans_time'], inplace=True)

In [ ]:
display(trev_transactions)

Distribution by weeks

In [ ]:
# separate days
trev_transactions['trans_time'] = trev_transactions['trans_time'].apply(keep_only_weeks_int)

trev_transactions_gr = trev_transactions.groupby(
    ['trans_time', 'gender', 'mcc_code'])['amount'].sum().reset_index().sort_values('trans_time')

trev_transactions_gr['amount'] = trev_transactions_gr['amount'].abs() 
# let's delete the last incomplete week so that the graph doesn't fall at the end 
trev_transactions_gr = trev_transactions.head(len(trev_transactions)-1)

# abs for non-grooped dataframe (used for scatter and box plots)
trev_transactions['amount'] = trev_transactions['amount'].abs() 

#remove extremums
def remove_extremums_trevel(amount: float) -> float:
    return (amount<100_000)*amount  

trev_transactions['amount'] = trev_transactions['amount'].apply(remove_extremums_trevel)


# change mcc codes to their russian names
trev_transactions_gr['mcc_code'] = trev_transactions_gr['mcc_code'].apply(mcc_code_means)


In [ ]:
#group categories; drop: city, gender, term_id, trans_type
trev_transactions_gr = trev_transactions_gr.groupby(['trans_time', 'mcc_code'])['amount'].sum().reset_index()

In [ ]:
fig_trevel_days_distr_bar = px.bar(
    # remove last not-full week
    trev_transactions_gr.head(len(trev_transactions_gr)-7), 
    x = 'trans_time', 
    y = 'amount', 
    color='mcc_code', 
    title='Distribution of travel expenses by weeks:', 
    labels={'trans_time':'week number'}, 
    template=main_template
)

fig_trevel_days_distr_bar.update_layout(
    legend_orientation='h', 
    legend_title = 'Category:', 
    legend_y=-0.25,
    legend_x = -0.1
)

fig_trevel_days_distr_bar.show()

Upgrade data (grouping, smothing, e.t.c)

In [ ]:
def group_data_trevel(mcc_code:str) -> str:
    group_data = {
        'Авиалинии, авиакомпании':'airline tickets',
        'Авиалинии, авиакомпании, нигде более не классифицированные':'airline tickets',
        'Жилье — отели, мотели, курорты': 'hotels', 
        'Отели, мотели, базы отдыха, сервисы бронирования':'hotels', 
        'Круизные линии':'cruises', 
        'Пассажирские железные перевозки':'railway tickets',
        'Прокат автомобилей':'car hire',
        'Агентства по аренде автомобилей':'car hire', 
        'Туристические агентства и организаторы экскурсий':'entertainments',
        'Туристические аттракционы и шоу':'entertainments'
    }
    return group_data[mcc_code]

trev_transactions_gr['mcc_code'] = trev_transactions_gr['mcc_code'].apply(group_data_trevel)

trev_transactions_gr = trev_transactions_gr.groupby(['trans_time', 'mcc_code'])['amount'].sum().reset_index()

In [ ]:
fig_adv_trevel_days_distr_bar = px.bar(
    # remove last not-full week
    trev_transactions_gr.head(len(trev_transactions_gr)-7), 
    x = 'trans_time', 
    y = 'amount', 
    color='mcc_code', 
    title='Distribution of travel expenses by weeks:', 
    labels={'trans_time':'week number'}, 
    template=main_template
)

fig_adv_trevel_days_distr_bar.update_layout(
    legend_orientation='h', 
    legend_title = 'Category:', 
    legend_y=-0.25,
    legend_x = -0.1
)

fig_adv_trevel_days_distr_bar.show()

we'll smooth out the graph by summing by two weeks instead of a week.

In [ ]:
def one_to_two_week(week_number:int) -> int:
    return (week_number//2) * 2

trev_transactions_gr_two = trev_transactions_gr.copy()
trev_transactions_gr_two['trans_time'] = trev_transactions_gr['trans_time'].apply(one_to_two_week)
trev_transactions_gr_two = trev_transactions_gr.groupby(['trans_time', 'mcc_code'])['amount'].sum().reset_index()

In [ ]:
fig_adv_trevel_days_distr_line = px.line(
    # remove last not-full week
    trev_transactions_gr_two.head(len(trev_transactions_gr)-7), 
    x = 'trans_time', 
    y = 'amount', 
    color='mcc_code', 
    title='Distribution of travel expenses by weeks:', 
    labels={'trans_time':'week number'}, 
    template=main_template,
    line_shape='spline'
)

fig_adv_trevel_days_distr_line.update_layout(
    legend_orientation='h', 
    legend_title = 'Category:', 
    legend_y=-0.25,
    legend_x = -0.1
)

fig_adv_trevel_days_distr_line.show()

icome - spent corrilation

In [ ]:

trev_users_total_spent = trev_transactions.reset_index().groupby(
    ['client_id', 'gender', 'mcc_code']
)['amount'].sum().reset_index().set_index('client_id').sort_index()
# group categories
trev_users_total_spent['mcc_code'] = trev_users_total_spent['mcc_code'].apply(mcc_code_means)
trev_users_total_spent['mcc_code'] = trev_users_total_spent['mcc_code'].apply(group_data_trevel)

trev_income_spend_correl = pd.merge(
    trev_users_total_spent,
    users_total_spend, 
    right_index=True, 
    left_index=True
).drop(columns=['gender_y']).rename(
    columns={
        'gender_x':'gender', 
        'amount_x':'spent',
        'amount_y':'income'
        })

# rename genders: from 0-1 to mele-female
trev_income_spend_correl['gender'] = np.where(
    trev_income_spend_correl['gender'], 'male', 'female')

# Let’s remove the extremes, as they prevent us from understanding the general trend
trev_income_spend_correl = trev_income_spend_correl.loc[
    (trev_income_spend_correl['income']<5_000_000) &
    (trev_income_spend_correl['spent']<50_000)
]

In [ ]:
fig_trev_income_scatter = px.scatter(
    trev_income_spend_correl,
    x = 'income', 
    y = 'spent', 
    color='gender', 
    title='The dependence of spending on treveling on total income', 
    template=main_template,
    log_x=False,
    
    
)
fig_trev_income_scatter.show()

conclusion: people who don't earn much money have a lot of money for travelling. (unexpectedly)

<h4>
Let’s try to make this graph more illustrative.
</h4>

    grouping by income (rounding), take spent median

In [ ]:
# grouping by income (rounding), take spent median

# round users income:
trev_income_spend_correl['income'] = trev_income_spend_correl['income'].apply(round_income)

# grouping by income:
trev_income_spend_correl = trev_income_spend_correl.reset_index().groupby(
    ['income', 'gender', 'mcc_code']
)['spent'].median().reset_index().sort_values('income')

# change mcc codes to names:

# cut the income range:
trev_income_spend_correl = trev_income_spend_correl.loc[trev_income_spend_correl['income']<10**6]


In [ ]:
fig_trevel_income_bar_pro = px.bar(
    trev_income_spend_correl,
    x = 'income', 
    y = 'spent', 
    color='mcc_code', 
    title='The dependence of spending on trevelling on total income', 
    template=main_template, 
)
fig_trevel_income_bar_pro.update_layout(
    legend_orientation='h', 
    legend_y = -0.25, 
    legend_title = 'Category:'
)
fig_trevel_income_bar_pro.add_trace(
    go.Line(
        x = trev_income_spend_correl.groupby(['income'])['spent'].sum().reset_index()['income'],
        y = trev_income_spend_correl.groupby(['income'])['spent'].sum().reset_index()['spent'], 
        name = 'category sum',
        line_shape = 'spline'
    )
)

fig_trevel_income_bar_pro.show()

Statistics by cities

In [ ]:
trev_city = trev_transactions.groupby(['trans_time', 'trans_city'])['amount'].sum().reset_index()

#make smother var
trev_city_smooth = trev_city.copy()
trev_city_smooth = trev_city_smooth.copy()
trev_city_smooth['trans_time'] = trev_city_smooth['trans_time'].apply(one_to_two_week)
trev_city_smooth = trev_city_smooth.groupby(['trans_time', 'trans_city'])['amount'].sum().reset_index()

In [ ]:
fig_trev_city_week_bar = px.bar(
    trev_city_smooth, 
    x = 'trans_time',
    y = 'amount',
    color='trans_city', 
    title="Travelling expenses in different cities of Russia", 
    labels={'trans_time': 'week_number'}, 
    template=main_template
)

fig_trev_city_week_bar.update_layout(
    legend_orientation='h', 
    legend_title = 'Category:', 
    legend_y=-0.25,
    legend_x = -0.1
)

fig_trev_city_week_bar.show()

In [ ]:
fig_trev_city_week_line = px.line(
    trev_city_smooth, 
    x = 'trans_time',
    y = 'amount',
    color='trans_city', 
    title="Travelling expenses in different cities of Russia", 
    labels={'trans_time': 'week_number'}, 
    template=main_template,
)

fig_trev_city_week_line.update_layout(
    legend_orientation='h', 
    legend_title = 'Category:', 
    legend_y=-0.25,
    legend_x = -0.1
)

fig_trev_city_week_line.show()

<h4>make box plot</h4>
<body>The travel expenses of inhabitants of various cities</body>

In [ ]:
fig_trev_city_box = px.box(
    trev_transactions, 
    x = 'trans_city', 
    y = 'amount', 
    title = 'The travel expenses of inhabitants of various cities', 
    labels={'trans_city':'city name'},
    template=main_template
)
fig_trev_city_box.update_layout( 
    yaxis=dict(
        range=[0,20_000]
    )
)

fig_trev_city_box.show()